<a href="https://colab.research.google.com/github/Infant-Joshva/AI_Agents-Learning_Materials/blob/main/Practical_Part/Pinecone_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U pinecone sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.8/736.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 26.0
    Uninstalling packaging-26.0:
      Successfully uninstalled packaging-26.0


In [1]:
import pinecone
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec

In [2]:
import os
from getpass import getpass

os.environ['PINECONE_API_KEY'] = getpass('Pinecone API Key:')

Pinecone API Key:··········


In [3]:
pc = Pinecone(os.environ['PINECONE_API_KEY'])
index = pc.Index("pinecone-demo")

model = SentenceTransformer('all-MiniLM-L6-v2')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
texts = ["Machine Learning is a part of Data Science", "AI is booming now"]
ids = ["doc1","doc2"]

In [6]:
embeddings = model.encode(texts).tolist()
print(embeddings)

[[-0.017961397767066956, -0.028813902288675308, 0.0722580999135971, 0.02247011289000511, 0.05489416420459747, -0.07317487895488739, -0.07226768136024475, -0.06925386935472488, -0.08156605064868927, 0.03309066593647003, -0.08220422267913818, 0.008342782035470009, 0.041486505419015884, -0.05226174741983414, -0.015167799778282642, 0.025804486125707626, -0.09679557383060455, 0.0075455051846802235, -0.04752638563513756, -0.07849260419607162, -0.029294125735759735, 0.0321166068315506, -0.06516110897064209, -0.003836645744740963, 0.028560783714056015, 0.022971853613853455, 0.0494820699095726, 0.0520198680460453, -0.01827901415526867, 0.025265591219067574, -0.03497053310275078, 0.005483528599143028, 0.01025312952697277, 0.05488679185509682, -0.07817774266004562, 0.05676455795764923, 0.010675245895981789, 0.04420778155326843, 0.04491645470261574, 0.019637828692793846, -0.03274101763963699, -0.04891262203454971, -0.030185801908373833, 0.005780423991382122, 0.0892607569694519, 0.07822404056787491

In [11]:
# (id, vector, metadata)
vectors = []

for i in range(len(texts)):
    vectors.append({
        "id": ids[i],
        "values": embeddings[i],
        "metadata": {"text": texts[i]}
    })

index.upsert(vectors=vectors)


UpsertResponse(upserted_count=2, _response_info={'raw_headers': {'date': 'Fri, 13 Feb 2026 10:41:16 GMT', 'content-type': 'application/json', 'content-length': '19', 'connection': 'keep-alive', 'x-pinecone-request-lsn': '1', 'x-pinecone-request-logical-size': '3163', 'x-pinecone-request-latency-ms': '269', 'x-envoy-upstream-service-time': '244', 'x-pinecone-response-duration-ms': '271', 'grpc-status': '0', 'server': 'envoy'}})

In [12]:
# query

query = "AI used in healthcare"
query_embedding = model.encode(query).tolist()

result = index.query(
    vector = query_embedding,
    top_k = 1,
    include_metadata = True
)

print(result)

QueryResponse(matches=[{'id': 'doc2',
 'metadata': {'text': 'AI is booming now'},
 'score': 0.546105444,
 'values': []}], namespace='', usage={'read_units': 1}, _response_info={'raw_headers': {'date': 'Fri, 13 Feb 2026 10:43:10 GMT', 'content-type': 'application/json', 'content-length': '151', 'connection': 'keep-alive', 'x-pinecone-max-indexed-lsn': '1', 'x-pinecone-request-latency-ms': '45', 'x-envoy-upstream-service-time': '46', 'x-pinecone-response-duration-ms': '47', 'grpc-status': '0', 'server': 'envoy'}})
